<a href="https://colab.research.google.com/github/Xaomomo/Unet-with-attention/blob/main/Linformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Linformer

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:08<00:00, 42.7MB/s]


In [ ]:
!unzip -q input.zip

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
from sklearn.model_selection import train_test_split
IDX=[x for x in range(len(metadat))]
train_index, val_index,_,_=train_test_split(
   IDX,metadat, test_size=0.05, random_state=2)

In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

##Linformer init

In [ ]:
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Keras-based multi-head attention layer."""


import collections
import math
import string

import numpy as np
import tensorflow as tf

from keras import constraints
from keras import initializers
from keras import regularizers
from keras.engine.base_layer import Layer
from keras.layers import activation
from keras.layers import core
from keras.layers import regularization
from keras.utils import tf_utils

# isort: off
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util.tf_export import keras_export

_CHR_IDX = string.ascii_lowercase

def _build_attention_equation(rank, attn_axes):
    """Builds einsum equations for the attention computation.
    Query, key, value inputs after projection are expected to have the shape as:
    `(bs, <non-attention dims>, <attention dims>, num_heads, channels)`.
    `bs` and `<non-attention dims>` are treated as `<batch dims>`.
    The attention operations can be generalized:
    (1) Query-key dot product:
    `(<batch dims>, <query attention dims>, num_heads, channels), (<batch dims>,
    <key attention dims>, num_heads, channels) -> (<batch dims>,
    num_heads, <query attention dims>, <key attention dims>)`
    (2) Combination:
    `(<batch dims>, num_heads, <query attention dims>, <key attention dims>),
    (<batch dims>, <value attention dims>, num_heads, channels) -> (<batch
    dims>, <query attention dims>, num_heads, channels)`
    Args:
      rank: Rank of query, key, value tensors.
      attn_axes: List/tuple of axes, `[-1, rank)`,
        that attention will be applied to.
    Returns:
      Einsum equations.
    """
    target_notation = _CHR_IDX[:rank]
    # `batch_dims` includes the head dim.
    batch_dims = tuple(np.delete(range(rank), attn_axes + (rank - 1,)))
    letter_offset = rank
    source_notation = ""
    for i in range(rank):
        if i in batch_dims or i == rank - 1:
            source_notation += target_notation[i]
        else:
            source_notation += _CHR_IDX[letter_offset]
            letter_offset += 1

    product_notation = "".join(
        [target_notation[i] for i in batch_dims]
        + [target_notation[i] for i in attn_axes]
        + [source_notation[i] for i in attn_axes]
    )
    dot_product_equation = "%s,%s->%s" % (
        source_notation,
        target_notation,
        product_notation,
    )
    attn_scores_rank = len(product_notation)
    combine_equation = "%s,%s->%s" % (
        product_notation,
        source_notation,
        target_notation,
    )
    return dot_product_equation, combine_equation, attn_scores_rank


def _build_proj_equation(free_dims, bound_dims, output_dims):
    """Builds an einsum equation for projections inside multi-head attention."""
    input_str = ""
    kernel_str = ""
    output_str = ""
    bias_axes = ""
    letter_offset = 0
    for i in range(free_dims):
        char = _CHR_IDX[i + letter_offset]
        input_str += char
        output_str += char

    letter_offset += free_dims
    for i in range(bound_dims):
        char = _CHR_IDX[i + letter_offset]
        input_str += char
        kernel_str += char

    letter_offset += bound_dims
    for i in range(output_dims):
        char = _CHR_IDX[i + letter_offset]
        kernel_str += char
        output_str += char
        bias_axes += char
    equation = f"{input_str},{kernel_str}->{output_str}"
    return equation, bias_axes, len(output_str)


def _get_output_shape(output_rank, known_last_dims):
    return [None] * (output_rank - len(known_last_dims)) + list(known_last_dims)

class Linformer(Layer):
    """MultiHeadAttention layer.
    This is an implementation of multi-headed attention as described in the
    paper "Attention is all you Need" (Vaswani et al., 2017).
    If `query`, `key,` `value` are the same, then
    this is self-attention. Each timestep in `query` attends to the
    corresponding sequence in `key`, and returns a fixed-width vector.
    This layer first projects `query`, `key` and `value`. These are
    (effectively) a list of tensors of length `num_attention_heads`, where the
    corresponding shapes are `(batch_size, <query dimensions>, key_dim)`,
    `(batch_size, <key/value dimensions>, key_dim)`,
    `(batch_size, <key/value dimensions>, value_dim)`.
    Then, the query and key tensors are dot-producted and scaled. These are
    softmaxed to obtain attention probabilities. The value tensors are then
    interpolated by these probabilities, then concatenated back to a single
    tensor.
    Finally, the result tensor with the last dimension as value_dim can take an
    linear projection and return.
    When using `MultiHeadAttention` inside a custom layer, the custom layer must
    implement its own `build()` method and call `MultiHeadAttention`'s
    `_build_from_signature()` there.
    This enables weights to be restored correctly when the model is loaded.
    Examples:
    Performs 1D cross-attention over two sequence inputs with an attention mask.
    Returns the additional attention weights over heads.
    >>> layer = MultiHeadAttention(num_heads=2, key_dim=2)
    >>> target = tf.keras.Input(shape=[8, 16])
    >>> source = tf.keras.Input(shape=[4, 16])
    >>> output_tensor, weights = layer(target, source,
    ...                                return_attention_scores=True)
    >>> print(output_tensor.shape)
    (None, 8, 16)
    >>> print(weights.shape)
    (None, 2, 8, 4)
    Performs 2D self-attention over a 5D input tensor on axes 2 and 3.
    >>> layer = MultiHeadAttention(
    ...     num_heads=2, key_dim=2, attention_axes=(2, 3))
    >>> input_tensor = tf.keras.Input(shape=[5, 3, 4, 16])
    >>> output_tensor = layer(input_tensor, input_tensor)
    >>> print(output_tensor.shape)
    (None, 5, 3, 4, 16)
    Args:
      num_heads: Number of attention heads.
      key_dim: Size of each attention head for query and key.
      value_dim: Size of each attention head for value.
      dropout: Dropout probability.
      use_bias: Boolean, whether the dense layers use bias vectors/matrices.
      output_shape: The expected shape of an output tensor, besides the batch
        and sequence dims. If not specified, projects back to the key feature
        dim.
      attention_axes: axes over which the attention is applied. `None` means
        attention over all axes, but batch, heads, and features.
      kernel_initializer: Initializer for dense layer kernels.
      bias_initializer: Initializer for dense layer biases.
      kernel_regularizer: Regularizer for dense layer kernels.
      bias_regularizer: Regularizer for dense layer biases.
      activity_regularizer: Regularizer for dense layer activity.
      kernel_constraint: Constraint for dense layer kernels.
      bias_constraint: Constraint for dense layer kernels.
    Call arguments:
      query: Query `Tensor` of shape `(B, T, dim)`.
      value: Value `Tensor` of shape `(B, S, dim)`.
      key: Optional key `Tensor` of shape `(B, S, dim)`. If not given, will use
        `value` for both `key` and `value`, which is the most common case.
      attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
        attention to certain positions. The boolean mask specifies which query
        elements can attend to which key elements, 1 indicates attention and 0
        indicates no attention. Broadcasting can happen for the missing batch
        dimensions and the head dimension.
      return_attention_scores: A boolean to indicate whether the output should
        be `(attention_output, attention_scores)` if `True`, or
        `attention_output` if `False`. Defaults to `False`.
      training: Python boolean indicating whether the layer should behave in
        training mode (adding dropout) or in inference mode (no dropout).
        Defaults to either using the training mode of the parent layer/model,
        or False (inference) if there is no parent layer.
      use_causal_mask: A boolean to indicate whether to apply a causal mask to
        prevent tokens from attending to future tokens (e.g., used in a decoder
        Transformer).
    Returns:
      attention_output: The result of the computation, of shape `(B, T, E)`,
        where `T` is for target sequence shapes and `E` is the query input last
        dimension if `output_shape` is `None`. Otherwise, the multi-head outputs
        are projected to the shape specified by `output_shape`.
      attention_scores: [Optional] multi-head attention coefficients over
        attention axes.
    """

    def __init__(
        self,
        num_heads,
        key_dim,
        value_dim=None,
        dropout=0.0,
        use_bias=True,
        output_shape=None,
        attention_axes=None,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        k_value=32,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.supports_masking = True
        self._num_heads = num_heads
        self._key_dim = key_dim
        self._value_dim = value_dim if value_dim else key_dim
        self._dropout = dropout
        self._use_bias = use_bias
        self.k_value = k_value
        self._output_shape = output_shape
        self._kernel_initializer = initializers.get(kernel_initializer)
        self._bias_initializer = initializers.get(bias_initializer)
        self._kernel_regularizer = regularizers.get(kernel_regularizer)
        self._bias_regularizer = regularizers.get(bias_regularizer)
        self._activity_regularizer = regularizers.get(activity_regularizer)
        self._kernel_constraint = constraints.get(kernel_constraint)
        self._bias_constraint = constraints.get(bias_constraint)
        if attention_axes is not None and not isinstance(
            attention_axes, collections.abc.Sized
        ):
            self._attention_axes = (attention_axes,)
        else:
            self._attention_axes = attention_axes
        self._built_from_signature = False
        self._query_shape, self._key_shape, self._value_shape = None, None, None

    def get_config(self):
        config = {
            "num_heads": self._num_heads,
            "key_dim": self._key_dim,
            "value_dim": self._value_dim,
            "dropout": self._dropout,
            "use_bias": self._use_bias,
            "output_shape": self._output_shape,
            "attention_axes": self._attention_axes,
            "kernel_initializer": initializers.serialize(
                self._kernel_initializer
            ),
            "bias_initializer": initializers.serialize(self._bias_initializer),
            "kernel_regularizer": regularizers.serialize(
                self._kernel_regularizer
            ),
            "bias_regularizer": regularizers.serialize(self._bias_regularizer),
            "activity_regularizer": regularizers.serialize(
                self._activity_regularizer
            ),
            "kernel_constraint": constraints.serialize(self._kernel_constraint),
            "bias_constraint": constraints.serialize(self._bias_constraint),
            "query_shape": self._query_shape,
            "key_shape": self._key_shape,
            "value_shape": self._value_shape,
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        # If the layer has a different build() function from the Keras default,
        # we need to trigger the customized build to create weights.
        query_shape = config.pop("query_shape")
        key_shape = config.pop("key_shape")
        value_shape = config.pop("value_shape")
        layer = cls(**config)
        if None in [query_shape, key_shape, value_shape]:
            logging.warning(
                "One of dimensions of the input shape is missing. It "
                "should have been memorized when the layer was serialized. "
                "%s is created without weights.",
                str(cls),
            )
        else:
            layer._build_from_signature(query_shape, value_shape, key_shape)
        return layer

    def _build_from_signature(self, query, value, key=None):
        """Builds layers and variables.
        Once the method is called, self._built_from_signature will be set to
        True.
        Args:
          query: Query tensor or TensorShape.
          value: Value tensor or TensorShape.
          key: Key tensor or TensorShape.
        """
        self._built_from_signature = True
        if hasattr(query, "shape"):
            self._query_shape = tf.TensorShape(query.shape)
        else:
            self._query_shape = tf.TensorShape(query)
        if hasattr(value, "shape"):
            self._value_shape = tf.TensorShape(value.shape)
        else:
            self._value_shape = tf.TensorShape(value)
        if key is None:
            self._key_shape = self._value_shape
        elif hasattr(key, "shape"):
            self._key_shape = tf.TensorShape(key.shape)
        else:
            self._key_shape = tf.TensorShape(key)

        # Any setup work performed only once should happen in an `init_scope`
        # to avoid creating symbolic Tensors that will later pollute any eager
        # operations.
        with tf_utils.maybe_init_scope(self):
            free_dims = self._query_shape.rank - 1
            einsum_equation, bias_axes, output_rank = _build_proj_equation(
                free_dims, bound_dims=1, output_dims=2
            )
            self._query_dense = core.EinsumDense(
                einsum_equation,
                output_shape=_get_output_shape(
                    output_rank - 1, [self._num_heads, self._key_dim]
                ),
                bias_axes=bias_axes if self._use_bias else None,
                name="query",
                **self._get_common_kwargs_for_sublayer(),
            )
            einsum_equation, bias_axes, output_rank = _build_proj_equation(
                self._key_shape.rank - 1, bound_dims=1, output_dims=2
            )
            self._key_dense = core.EinsumDense(
                einsum_equation,
                output_shape=_get_output_shape(
                    output_rank - 1, [self._num_heads, self._key_dim]
                ),
                bias_axes=bias_axes if self._use_bias else None,
                name="key",
                **self._get_common_kwargs_for_sublayer(),
            )
            self._proj=tf.keras.layers.EinsumDense("bjhd,kj->bkhd",output_shape=(self.k_value,None,None))

            einsum_equation, bias_axes, output_rank = _build_proj_equation(
                self._value_shape.rank - 1, bound_dims=1, output_dims=2
            )
            self._value_dense = core.EinsumDense(
                einsum_equation,
                output_shape=_get_output_shape(
                    output_rank - 1, [self._num_heads, self._value_dim]
                ),
                bias_axes=bias_axes if self._use_bias else None,
                name="value",
                **self._get_common_kwargs_for_sublayer(),
            )

            # Builds the attention computations for multi-head dot product
            # attention.  These computations could be wrapped into the keras
            # attention layer once it supports mult-head einsum computations.
            self._build_attention(output_rank)
            self._output_dense = self._make_output_dense(
                free_dims,
                self._get_common_kwargs_for_sublayer(),
                "attention_output",
            )

    def _get_common_kwargs_for_sublayer(self):
        common_kwargs = dict(
            kernel_regularizer=self._kernel_regularizer,
            bias_regularizer=self._bias_regularizer,
            activity_regularizer=self._activity_regularizer,
            kernel_constraint=self._kernel_constraint,
            bias_constraint=self._bias_constraint,
        )
        # Create new clone of kernel/bias initializer, so that we don't reuse
        # the initializer instance, which could lead to same init value since
        # initializer is stateless.
        kernel_initializer = self._kernel_initializer.__class__.from_config(
            self._kernel_initializer.get_config()
        )
        bias_initializer = self._bias_initializer.__class__.from_config(
            self._bias_initializer.get_config()
        )
        common_kwargs["kernel_initializer"] = kernel_initializer
        common_kwargs["bias_initializer"] = bias_initializer
        return common_kwargs

    def _make_output_dense(self, free_dims, common_kwargs, name=None):
        """Builds the output projection matrix.
        Args:
          free_dims: Number of free dimensions for einsum equation building.
          common_kwargs: Common keyword arguments for einsum layer.
          name: Name for the projection layer.
        Returns:
          Projection layer.
        """
        if self._output_shape:
            if not isinstance(self._output_shape, collections.abc.Sized):
                output_shape = [self._output_shape]
            else:
                output_shape = self._output_shape
        else:
            output_shape = [self._query_shape[-1]]
        einsum_equation, bias_axes, output_rank = _build_proj_equation(
            free_dims, bound_dims=2, output_dims=len(output_shape)
        )
        return core.EinsumDense(
            einsum_equation,
            output_shape=_get_output_shape(output_rank - 1, output_shape),
            bias_axes=bias_axes if self._use_bias else None,
            name=name,
            **common_kwargs,
        )

    def _build_attention(self, rank):
        """Builds multi-head dot-product attention computations.
        This function builds attributes necessary for `_compute_attention` to
        customize attention computation to replace the default dot-product
        attention.
        Args:
          rank: the rank of query, key, value tensors.
        """
        if self._attention_axes is None:
            self._attention_axes = tuple(range(1, rank - 2))
        else:
            self._attention_axes = tuple(self._attention_axes)
        (
            self._dot_product_equation,
            self._combine_equation,
            attn_scores_rank,
        ) = _build_attention_equation(rank, attn_axes=self._attention_axes)
        norm_axes = tuple(
            range(
                attn_scores_rank - len(self._attention_axes), attn_scores_rank
            )
        )
        self._softmax = activation.Softmax(axis=norm_axes)
        self._dropout_layer = regularization.Dropout(rate=self._dropout)

    def _masked_softmax(self, attention_scores, attention_mask=None):
        # Normalize the attention scores to probabilities.
        # `attention_scores` = [B, N, T, S]
        if attention_mask is not None:
            # The expand dim happens starting from the `num_heads` dimension,
            # (<batch_dims>, num_heads, <query_attention_dims,
            # key_attention_dims>)
            mask_expansion_axis = -len(self._attention_axes) * 2 - 1
            for _ in range(
                len(attention_scores.shape) - len(attention_mask.shape)
            ):
                attention_mask = tf.expand_dims(
                    attention_mask, axis=mask_expansion_axis
                )
        return self._softmax(attention_scores, attention_mask)

    def _compute_attention(
        self, query, key, value, attention_mask=None, training=None
    ):
        """Applies Dot-product attention with query, key, value tensors.
        This function defines the computation inside `call` with projected
        multi-head Q, K, V inputs. Users can override this function for
        customized attention implementation.
        Args:
          query: Projected query `Tensor` of shape `(B, T, N, key_dim)`.
          key: Projected key `Tensor` of shape `(B, S, N, key_dim)`.
          value: Projected value `Tensor` of shape `(B, S, N, value_dim)`.
          attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
            attention to certain positions. It is generally not needed if the
            `query` and `value` (and/or `key`) are masked.
          training: Python boolean indicating whether the layer should behave in
            training mode (adding dropout) or in inference mode (doing nothing).
        Returns:
          attention_output: Multi-headed outputs of attention computation.
          attention_scores: Multi-headed attention weights.
        """
        # Note: Applying scalar multiply at the smaller end of einsum improves
        # XLA performance, but may introduce slight numeric differences in
        # the Transformer attention head.
        query = tf.multiply(query, 1.0 / math.sqrt(float(self._key_dim)))
        # Take the dot product between "query" and "key" to get the raw
        # attention scores.
        attention_scores = tf.einsum(self._dot_product_equation, key, query)
        attention_scores = self._masked_softmax(
            attention_scores, attention_mask
        )
        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_scores_dropout = self._dropout_layer(
            attention_scores, training=training
        )

        # `context_layer` = [B, T, N, H]
        attention_output = tf.einsum(
            self._combine_equation, attention_scores_dropout, value
        )
        #import sys
        #sys.exit()
        return attention_output, attention_scores

    def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
        use_causal_mask=False,
    ):
        attention_mask = self._compute_attention_mask(
            query,
            value,
            key=key,
            attention_mask=attention_mask,
            use_causal_mask=use_causal_mask,
        )

        if not self._built_from_signature:
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query_is_ragged = isinstance(query, tf.RaggedTensor)
        if query_is_ragged:
            query_lengths = query.nested_row_lengths()
            query = query.to_tensor()

        key_is_ragged = isinstance(key, tf.RaggedTensor)
        value_is_ragged = isinstance(value, tf.RaggedTensor)
        if key_is_ragged and value_is_ragged:
            # Ensure they have the same shape.
            bounding_shape = tf.math.maximum(
                key.bounding_shape(), value.bounding_shape()
            )
            key = key.to_tensor(shape=bounding_shape)
            value = value.to_tensor(shape=bounding_shape)
        elif key_is_ragged:
            key = key.to_tensor(shape=tf.shape(value))
        elif value_is_ragged:
            value = value.to_tensor(shape=tf.shape(key))

        #   N = `num_attention_heads`
        #   H = `size_per_head`
        # `query` = [B, T, N ,H]
        query = self._query_dense(query)
        # `key` = [B, S, N, H]
        key = self._key_dense(key)
        key = self._proj(key)
        # `value` = [B, S, N, H]
        value = self._value_dense(value)
        value = self._proj(value)
        attention_output, attention_scores = self._compute_attention(
            query, key, value, attention_mask, training
        )
        attention_output = self._output_dense(attention_output)

        if query_is_ragged:
            attention_output = tf.RaggedTensor.from_tensor(
                attention_output, lengths=query_lengths
            )

        if return_attention_scores:
            return attention_output, attention_scores
        return attention_output

    def _compute_attention_mask(
        self, query, value, key=None, attention_mask=None, use_causal_mask=False
    ):
        """Computes the attention mask, using the Keras masks of the inputs.
        * The `query`'s mask is reshaped from [B, T] to [B, T, 1].
        * The `value`'s mask is reshaped from [B, S] to [B, 1, S].
        * The `key`'s mask is reshaped from [B, S] to [B, 1, S]. The `key`'s
          mask is ignored if `key` is `None` or if `key is value`.
        * If `use_causal_mask=True`, then the causal mask is computed. Its shape
          is [1, T, S].
        All defined masks are merged using a logical AND operation (`&`).
        In general, if the `query` and `value` are masked, then there is no need
        to define the `attention_mask`.
        Args:
          query: Projected query `Tensor` of shape `(B, T, N, key_dim)`.
          key: Projected key `Tensor` of shape `(B, T, N, key_dim)`.
          value: Projected value `Tensor` of shape `(B, T, N, value_dim)`.
          attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
            attention to certain positions.
          use_causal_mask: A boolean to indicate whether to apply a causal mask
            to prevent tokens from attending to future tokens (e.g., used in a
            decoder Transformer).
        Returns:
          attention_mask: a boolean mask of shape `(B, T, S)`, that prevents
            attention to certain positions, based on the Keras masks of the
            `query`, `key`, `value`, and `attention_mask` tensors, and the
            causal mask if `use_causal_mask=True`.
        """
        query_mask = getattr(query, "_keras_mask", None)
        value_mask = getattr(value, "_keras_mask", None)
        key_mask = getattr(key, "_keras_mask", None)
        auto_mask = None
        if query_mask is not None:
            query_mask = tf.cast(query_mask, tf.bool)  # defensive casting
            # B = batch size, T = max query length
            auto_mask = query_mask[:, :, tf.newaxis]  # shape is [B, T, 1]
        if value_mask is not None:
            value_mask = tf.cast(value_mask, tf.bool)  # defensive casting
            # B = batch size, S == max value length
            mask = value_mask[:, tf.newaxis, :]  # shape is [B, 1, S]
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if key_mask is not None:
            key_mask = tf.cast(key_mask, tf.bool)  # defensive casting
            # B == batch size, S == max key length == max value length
            mask = key_mask[:, tf.newaxis, :]  # shape is [B, 1, S]
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if use_causal_mask:
            # the shape of the causal mask is [1, T, S]
            mask = self._compute_causal_mask(query, value)
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if auto_mask is not None:
            # merge attention_mask & automatic mask, to shape [B, T, S]
            attention_mask = (
                auto_mask
                if attention_mask is None
                else tf.cast(attention_mask, bool) & auto_mask
            )
        return attention_mask

    def _compute_causal_mask(self, query, value=None):
        """Computes a causal mask (e.g., for masked self-attention layers).
        For example, if query and value both contain sequences of length 4,
        this function returns a boolean `Tensor` equal to:
        ```
        [[[True,  False, False, False],
          [True,  True,  False, False],
          [True,  True,  True,  False],
          [True,  True,  True,  True]]]
        ```
        Args:
          query: query `Tensor` of shape `(B, T, ...)`.
          value: value `Tensor` of shape `(B, S, ...)` (optional, defaults to
          query).
        Returns:
          mask: a boolean `Tensor` of shape [1, T, S] containing a lower
                triangular matrix of shape [T, S].
        """
        q_seq_length = tf.shape(query)[1]
        v_seq_length = q_seq_length if value is None else tf.shape(value)[1]
        return tf.linalg.band_part(  # creates a lower triangular matrix
            tf.ones((1, q_seq_length, v_seq_length), tf.bool), -1, 0
        )

    def compute_output_shape(self, query_shape, value_shape, key_shape=None):

        if key_shape is None:
            key_shape = value_shape

        query_shape = tf.TensorShape(query_shape)
        value_shape = tf.TensorShape(value_shape)
        key_shape = tf.TensorShape(key_shape)

        if query_shape[-1] != value_shape[-1]:
            raise ValueError(
                "The last dimension of `query_shape` and `value_shape` "
                f"must be equal, but are {query_shape[-1]}, {value_shape[-1]}. "
                "Received: query_shape={query_shape}, value_shape={value_shape}"
            )

        if value_shape[1:-1] != key_shape[1:-1]:
            raise ValueError(
                "All dimensions of `value` and `key`, except the last one, "
                f"must be equal. Received {value_shape} and "
                f"{key_shape}"
            )

        if self._output_shape:
            return query_shape[:-1].concatenate(self._output_shape)

        return query_shape

##Linformer as keras multihead subclass

In [ ]:
import collections
import math
import string
import numpy as np
import tensorflow as tf
from keras.layers.attention.multi_head_attention import MultiHeadAttention
from keras.utils import tf_utils




class Linformer(MultiHeadAttention):
  def __init__(self, *args, **kwargs):
    self.k_value = kwargs.pop('k_value', 128)
    super().__init__(*args, **kwargs)
  def _build_from_signature(self, query, value, key=None):
    with tf_utils.maybe_init_scope(self):
      self._proj=tf.keras.layers.EinsumDense("bjhd,kj->bkhd",output_shape=(self.k_value,None,None))
    super()._build_from_signature(query, value, key=None)

  def call(
        self,
        query,
        value,
        key=None,
        attention_mask=None,
        return_attention_scores=False,
        training=None,
        use_causal_mask=False,
        ):
        attention_mask = self._compute_attention_mask(
            query,
            value,
            key=key,
            attention_mask=attention_mask,
            use_causal_mask=use_causal_mask,
        )

        if not self._built_from_signature:
            self._build_from_signature(query=query, value=value, key=key)
        if key is None:
            key = value

        query_is_ragged = isinstance(query, tf.RaggedTensor)
        if query_is_ragged:
            query_lengths = query.nested_row_lengths()
            query = query.to_tensor()

        key_is_ragged = isinstance(key, tf.RaggedTensor)
        value_is_ragged = isinstance(value, tf.RaggedTensor)
        if key_is_ragged and value_is_ragged:
            # Ensure they have the same shape.
            bounding_shape = tf.math.maximum(
                key.bounding_shape(), value.bounding_shape()
            )
            key = key.to_tensor(shape=bounding_shape)
            value = value.to_tensor(shape=bounding_shape)
        elif key_is_ragged:
            key = key.to_tensor(shape=tf.shape(value))
        elif value_is_ragged:
            value = value.to_tensor(shape=tf.shape(key))

        #   N = `num_attention_heads`
        #   H = `size_per_head`
        # `query` = [B, T, N ,H]
        query = self._query_dense(query)
        # `key` = [B, S, N, H]
        key = self._key_dense(key)
        key = self._proj(key)
        # `value` = [B, S, N, H]
        value = self._value_dense(value)
        value = self._proj(value)
        attention_output, attention_scores = self._compute_attention(
            query, key, value, attention_mask, training
        )
        attention_output = self._output_dense(attention_output)

        if query_is_ragged:
            attention_output = tf.RaggedTensor.from_tensor(
                attention_output, lengths=query_lengths
        )

        if return_attention_scores:
            return attention_output, attention_scores
        return attention_output

##Patch wise

In [ ]:
from keras.layers import Layer
import keras.backend as K
import torch.nn as nn
import tensorflow as tf

In [ ]:
def DiceMetric(y_true, y_pred):
  smooth=1e-6
  gama=2
  y_true, y_pred = tf.cast(
      y_true, dtype=tf.float32), tf.cast(y_pred, tf.float32)
  nominator = 2 * \
      tf.reduce_sum(tf.multiply(y_pred, y_true)) + smooth
  denominator = tf.reduce_sum(
      y_pred ** gama) + tf.reduce_sum(y_true ** gama) + smooth
  result = tf.divide(nominator, denominator)
  return result
def DiceLoss(y_true, y_pred):
      result= 1- DiceMetric(y_true, y_pred)
      return result

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [ ]:
def dice_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      #None,128,128,1
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      #None,128,128
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += dice_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def iou_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += iou_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def conv_block(X,f,d=0.1,group=1):
  c = tf.keras.layers.Conv2D(f[0], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',groups=group)(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), kernel_initializer='he_normal', padding='same', groups=group)(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  s = tf.keras.layers.Conv2D(f[1], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([c,s])
  c = tf.keras.layers.ReLU()(c)
  return c,c

In [ ]:
import sys

In [ ]:
def vit(X,n_heads=16,psize=16,k=256):
  npatch=int(X.shape[1]/psize)
  for i in range(npatch):
    st=i*psize
    end=st+psize
    for j in range(npatch):
      st2=j*psize
      end2=st2+psize
      patch= X[:,st:end,st2:end2,:]
      s1=patch.shape[1]
      s2=patch.shape[2]
      s3=patch.shape[3]
      flat=s1*s2
      kdim=int(s3/n_heads)
      patch= tf.reshape(patch,[-1,flat,s3])
      pm=Linformer(num_heads=n_heads,key_dim=kdim,attention_axes=(1),dropout=0.2,k_value=k)(patch,patch)
      pm=tf.reshape(pm,[-1,s1,s2,s3])
      if j==0:
        A=pm
      else:
        A=tf.concat([A,pm],2)
    if i==0:
      V=A
    else:
      V=tf.concat([V,A],1)
  return V



Vit 2, with dropout

In [ ]:
def vit(X,n_heads=16,psize=16):
  npatch=int(X.shape[1]/psize)
  temp=0.5
  for i in range(npatch):
    st=i*psize
    end=st+psize
    for j in range(npatch):
      st2=j*psize
      end2=st2+psize
      patch= X[:,st:end,st2:end2,:]
      random.uniform(0,1)
      if random.uniform(0,1)> temp:
        pm=tf.keras.layers.MultiHeadAttention(num_heads=n_heads, key_dim=3, attention_axes=(1, 2))(patch,patch)
      else:
        pm=patch
      if j==0:
        A=pm
      else:
        A=tf.concat([A,pm],2)
    if i==0:
      V=A
    else:
      V=tf.concat([V,A],1)
  return V

### Vit

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path
  s1=z4.shape[1]
  s2=z4.shape[2]
  s3=z4.shape[3]
  flat=s1*s2
  z4= tf.reshape(z4,[-1,flat,s3])
  m1= Linformer(num_heads=nheads, key_dim=int(s3/nheads), attention_axes=(1))(z4,z4)
  m1=tf.reshape(m1,[-1,s1,s2,s3])
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= vit(z3,nheads,sizep)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= vit(z2,nheads,sizep)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= vit(z1,nheads,sizep)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###Linformer 2

In [ ]:
def Linformer_lay(M,nheads,k=128):
  s1=M.shape[1]
  s2=M.shape[2]
  s3=M.shape[3]
  flat=s1*s2
  M= tf.reshape(M,[-1,flat,s3])
  m= Linformer(num_heads=nheads, key_dim=int(s3/nheads), attention_axes=(1),k_value=k)(M,M)
  m=tf.reshape(m,[-1,s1,s2,s3])
  return m

In [ ]:
import tensorflow as tf

def create_model():
  nheads=4
  sizep=32
  gr=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=gr)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=gr)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=gr)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,z5 = conv_block(p4,[256,256],0.3,group=gr)

  #Expansive path

  m1= Linformer_lay(z4,nheads)
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, m1])
  c6,z6 = conv_block(u6,[128,128],0.2,group=gr)

  m2= Linformer_lay(z3,nheads)
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, m2])
  c7,z7 = conv_block(u7,[64,64],0.2,group=gr)

  m3= Linformer_lay(z2,nheads)
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, m3])
  c8,z8 = conv_block(u8,[32,32],group=gr)

  m4= Linformer_lay(z1,nheads)
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, m4], axis=3)
  c9,_ = conv_block(u9,[16,16],group=gr)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

###Test

In [ ]:
model=create_model()
model.fit(X_train,Y_train,batch_size=16,epochs=25)

Epoch 1/25
214/214 [==============================] - 132s 344ms/step - loss: 0.5585 - accuracy: 0.8376 - iou_coef_multilabel: 0.2579 - dice_coef_multilabel: 0.3317
Epoch 2/25
214/214 [==============================] - 69s 321ms/step - loss: 0.0742 - accuracy: 0.9855 - iou_coef_multilabel: 0.4980 - dice_coef_multilabel: 0.5614
Epoch 3/25
214/214 [==============================] - 69s 321ms/step - loss: 0.0364 - accuracy: 0.9916 - iou_coef_multilabel: 0.5818 - dice_coef_multilabel: 0.6356
Epoch 4/25
214/214 [==============================] - 69s 321ms/step - loss: 0.0260 - accuracy: 0.9931 - iou_coef_multilabel: 0.6200 - dice_coef_multilabel: 0.6686
Epoch 5/25
214/214 [==============================] - 69s 321ms/step - loss: 0.0208 - accuracy: 0.9940 - iou_coef_multilabel: 0.6454 - dice_coef_multilabel: 0.6904
Epoch 6/25
214/214 [==============================] - 69s 321ms/step - loss: 0.0186 - accuracy: 0.9943 - iou_coef_multilabel: 0.6613 - dice_coef_multilabel: 0.7040
Epoch 7/25
214/

In [ ]:
scores= model.evaluate(X_test, Y_test, verbose=0)
print(f'Score for fold {0}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')

Score for fold 0: loss of 0.11757280677556992; accuracy of 96.79576754570007% iou_coef_multilabel of 71.6090202331543% dice_coef_multilabel of 79.81376647949219%


Patch size=32 k=32:

Score for fold 0: loss of 0.02660856954753399; accuracy of 99.21053647994995% iou_coef_multilabel of 73.97453188896179% dice_coef_multilabel of 77.59955525398254%

No Patch k=128:

Score for fold 0: loss of 0.03675605729222298; accuracy of 98.8860547542572% iou_coef_multilabel of 69.78949904441833% dice_coef_multilabel of 74.2724359035492%

Patch size=32 k=256:
Score for fold 0: loss of 0.11757280677556992; accuracy of 96.79576754570007% iou_coef_multilabel of 71.6090202331543% dice_coef_multilabel of 79.81376647949219%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

####Patch test

k=32

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.019922519102692604; accuracy of 99.3967354297638% iou_coef_multilabel of 77.20364928245544% dice_coef_multilabel of 80.38443922996521%
Score for fold 2: loss of 0.032541245222091675; accuracy of 99.03221726417542% iou_coef_multilabel of 77.34413743019104% dice_coef_multilabel of 81.7846953868866%
Score for fold 3: loss of 0.032382525503635406; accuracy of 99.13042187690735% iou_coef_multilabel of 76.8735408782959% dice_coef_multilabel of 81.07832670211792%
Score for fold 4: loss of 0.0299893356859684; accuracy of 98.92753958702087% iou_coef_multilabel of 75.21778345108032% dice_coef_multilabel of 80.6958019733429%
Score for fold 5: loss of 0.022455349564552307; accuracy of 99.26480650901794% iou_coef_multilabel of 74.87989068031311% dice_coef_multilabel of 78.63270044326782%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01698482222855091; accuracy of 99.46913719177246% iou_coef_multilabel of 80.59046268463135% dice_coef_multilabel of 83.47237706184387%
Score for fold 2: loss of 0.03444297984242439; accuracy of 99.04292821884155% iou_coef_multilabel of 80.84890246391296% dice_coef_multilabel of 85.20692586898804%
Score for fold 3: loss of 0.03221513703465462; accuracy of 99.17513728141785% iou_coef_multilabel of 80.00396490097046% dice_coef_multilabel of 84.08617377281189%
Score for fold 4: loss of 0.030498819425702095; accuracy of 99.04159307479858% iou_coef_multilabel of 80.91708421707153% dice_coef_multilabel of 85.87648272514343%
Score for fold 5: loss of 0.028706222772598267; accuracy of 99.25541877746582% iou_coef_multilabel of 79.54670190811157% dice_coef_multilabel of 82.86161422729492%


k=256

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

KeyboardInterrupt: ignored

####No Patch test

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.014879913069307804; accuracy of 99.50977563858032% iou_coef_multilabel of 75.01700520515442% dice_coef_multilabel of 78.04171442985535%
Score for fold 2: loss of 0.03799952194094658; accuracy of 98.88894557952881% iou_coef_multilabel of 79.5952320098877% dice_coef_multilabel of 83.83917808532715%
Score for fold 3: loss of 0.027269238606095314; accuracy of 99.2344617843628% iou_coef_multilabel of 78.04872393608093% dice_coef_multilabel of 82.01154470443726%
Score for fold 4: loss of 0.035040128976106644; accuracy of 98.67711067199707% iou_coef_multilabel of 70.38484811782837% dice_coef_multilabel of 76.41092538833618%
Score for fold 5: loss of 0.04386671259999275; accuracy of 99.13702011108398% iou_coef_multilabel of 76.85499787330627% dice_coef_multilabel of 80.35175204277039%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.018669821321964264; accuracy of 99.44900274276733% iou_coef_multilabel of 82.00370669364929% dice_coef_multilabel of 84.69013571739197%
Score for fold 2: loss of 0.03746144101023674; accuracy of 98.98282289505005% iou_coef_multilabel of 79.05458807945251% dice_coef_multilabel of 83.39856266975403%
Score for fold 3: loss of 0.029935374855995178; accuracy of 99.2314100265503% iou_coef_multilabel of 81.18868470191956% dice_coef_multilabel of 85.30572056770325%
Score for fold 4: loss of 0.028801847249269485; accuracy of 99.00671243667603% iou_coef_multilabel of 81.22592568397522% dice_coef_multilabel of 86.3890290260315%
Score for fold 5: loss of 0.029414121061563492; accuracy of 99.23427700996399% iou_coef_multilabel of 77.37159729003906% dice_coef_multilabel of 80.87003231048584%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01531380508095026; accuracy of 99.50594902038574% iou_coef_multilabel of 80.83136677742004% dice_coef_multilabel of 83.69719982147217%
Score for fold 2: loss of 0.04106637462973595; accuracy of 98.89675974845886% iou_coef_multilabel of 81.13894462585449% dice_coef_multilabel of 85.50359010696411%
Score for fold 3: loss of 0.031158242374658585; accuracy of 99.2102563381195% iou_coef_multilabel of 80.927973985672% dice_coef_multilabel of 85.0109875202179%
Score for fold 4: loss of 0.028589071705937386; accuracy of 99.01348948478699% iou_coef_multilabel of 79.61382269859314% dice_coef_multilabel of 84.92236733436584%
Score for fold 5: loss of 0.02963246963918209; accuracy of 99.15727376937866% iou_coef_multilabel of 75.42580366134644% dice_coef_multilabel of 78.91713976860046%


##Model data augmented kfold

In [ ]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("vertical"),
  tf.keras.layers.RandomRotation(0.1),
])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  train_data = tf.data.Dataset.from_tensor_slices((X_fold, Y_fold))
  # Apply the data augmentation pipeline to both X_train and Y_train
  augmented_train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x, y: x)
  Y_train_augmented = augmented_train_data.map(lambda x, y: y)

  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = np.asarray(list(Y_train_augmented.as_numpy_iterator()))

  X_train_combined = np.concatenate([X_fold, X_train_augmented], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.030574526637792587; accuracy of 99.18301105499268% iou_coef_multilabel of 77.91773676872253% dice_coef_multilabel of 81.18282556533813%
Score for fold 2: loss of 0.0644448772072792; accuracy of 98.29415082931519% iou_coef_multilabel of 74.46072101593018% dice_coef_multilabel of 79.81048822402954%
Score for fold 3: loss of 0.06618913263082504; accuracy of 98.34594130516052% iou_coef_multilabel of 73.79118204116821% dice_coef_multilabel of 78.7887454032898%
Score for fold 4: loss of 0.05361005291342735; accuracy of 98.4992504119873% iou_coef_multilabel of 76.66850090026855% dice_coef_multilabel of 82.41685032844543%
Score for fold 5: loss of 0.06054575368762016; accuracy of 98.66406321525574% iou_coef_multilabel of 70.25530338287354% dice_coef_multilabel of 74.33005571365356%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  train_data = tf.data.Dataset.from_tensor_slices((X_fold, Y_fold))
  # Apply the data augmentation pipeline to both X_train and Y_train
  augmented_train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x, y: x)
  Y_train_augmented = augmented_train_data.map(lambda x, y: y)

  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = np.asarray(list(Y_train_augmented.as_numpy_iterator()))

  X_train_combined = np.concatenate([X_fold, X_train_augmented], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.04008716717362404; accuracy of 98.93805980682373% iou_coef_multilabel of 79.03890013694763% dice_coef_multilabel of 82.59339332580566%
Score for fold 2: loss of 0.0618879459798336; accuracy of 98.21731448173523% iou_coef_multilabel of 75.36373138427734% dice_coef_multilabel of 80.78001737594604%
Score for fold 3: loss of 0.05857839807868004; accuracy of 98.21037650108337% iou_coef_multilabel of 69.53166723251343% dice_coef_multilabel of 75.00591278076172%
Score for fold 4: loss of 0.05203528329730034; accuracy of 98.3975350856781% iou_coef_multilabel of 70.11039853096008% dice_coef_multilabel of 76.27233266830444%
Score for fold 5: loss of 0.05389085412025452; accuracy of 98.55652451515198% iou_coef_multilabel of 70.12524604797363% dice_coef_multilabel of 74.36990141868591%


##Model data augmented kfold Fixed

In [ ]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("vertical"),
  tf.keras.layers.RandomRotation(0.1),
])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  X__2 = np.concatenate([X_fold, Y_fold], axis=-1)
  X__2 = tf.data.Dataset.from_tensor_slices((X__2))
  augmented_train_data = X__2.map(lambda x: data_augmentation(x, training=True))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x: x)
  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = X_train_augmented[:,:,:,1:5]
  X_train_augmented = X_train_augmented[:,:,:,0]

  X_train_combined = np.concatenate([X_fold, X_train_augmented[:,:,:,None]], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01687796600162983; accuracy of 99.54043626785278% iou_coef_multilabel of 85.70728302001953% dice_coef_multilabel of 88.2942795753479%
Score for fold 2: loss of 0.03296380117535591; accuracy of 99.04602766036987% iou_coef_multilabel of 80.73644638061523% dice_coef_multilabel of 85.05041003227234%
Score for fold 3: loss of 0.03169004246592522; accuracy of 99.29794073104858% iou_coef_multilabel of 82.87500143051147% dice_coef_multilabel of 86.73526048660278%
Score for fold 4: loss of 0.02590073086321354; accuracy of 99.14000630378723% iou_coef_multilabel of 82.01050758361816% dice_coef_multilabel of 87.04196810722351%
Score for fold 5: loss of 0.023470886051654816; accuracy of 99.27202463150024% iou_coef_multilabel of 78.13495993614197% dice_coef_multilabel of 81.73242211341858%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  X__2 = np.concatenate([X_fold, Y_fold], axis=-1)
  X__2 = tf.data.Dataset.from_tensor_slices((X__2))
  augmented_train_data = X__2.map(lambda x: data_augmentation(x, training=True))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x: x)
  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = X_train_augmented[:,:,:,1:5]
  X_train_augmented = X_train_augmented[:,:,:,0]

  X_train_combined = np.concatenate([X_fold, X_train_augmented[:,:,:,None]], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.015453373081982136; accuracy of 99.50757026672363% iou_coef_multilabel of 82.24123120307922% dice_coef_multilabel of 85.04875302314758%
Score for fold 2: loss of 0.04136096313595772; accuracy of 98.85668754577637% iou_coef_multilabel of 78.5542905330658% dice_coef_multilabel of 82.86908864974976%
Score for fold 3: loss of 0.026296231895685196; accuracy of 99.25772547721863% iou_coef_multilabel of 79.66246604919434% dice_coef_multilabel of 83.86256098747253%
Score for fold 4: loss of 0.023807788267731667; accuracy of 99.12872910499573% iou_coef_multilabel of 77.16266512870789% dice_coef_multilabel of 82.68983960151672%
Score for fold 5: loss of 0.04154125601053238; accuracy of 99.09101128578186% iou_coef_multilabel of 78.13253998756409% dice_coef_multilabel of 81.70059323310852%


Score for fold 1: loss of 0.015777496621012688; accuracy of 99.4891345500946% iou_coef_multilabel of 83.78176689147949% dice_coef_multilabel of 86.61818504333496%

Score for fold 2: loss of 0.03494168445467949; accuracy of 98.98154139518738% iou_coef_multilabel of 81.28145337104797% dice_coef_multilabel of 85.57209968566895%

Score for fold 3: loss of 0.03187831863760948; accuracy of 99.23022985458374% iou_coef_multilabel of 78.91602516174316% dice_coef_multilabel of 82.88114666938782%